In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [148]:
data = pd.read_csv("../input/fetal-health-classification/fetal_health.csv")
data_copy = data.copy()


In [149]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [150]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)


In [151]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.30, random_state = 109)

In [152]:
from sklearn.svm import SVC

In [153]:
def plot_points(features, labels):
    X = np.array(features)
    y = np.array(labels)
    spam = X[np.argwhere(y==1)]
    ham = X[np.argwhere(y==2)]
    star = X[np.argwhere(y==3)]
    plt.scatter([s[0][0] for s in spam],
                   [s[0][1] for s in spam],
                   s = 25,
                   color = 'cyan',
                   edgecolor = 'k',
                   marker = '^')
    plt.scatter([s[0][0] for s in ham],
                   [s[0][1] for s in ham],
                   s = 25,
                   color = 'red',
                   edgecolor = 'k',
                   marker = 's')
    plt.scatter([s[0][0] for s in star],
                   [s[0][1] for s in star],
                   s = 25,
                   color = 'yellow',
                   edgecolor = 'k',
                   marker = 's')
    plt.xlabel('X_test')
    plt.ylabel('Y_test')
    plt.legend(['label 2','label 1','label 0'])

def plot_model(X, y, model):
    X = np.array(X)
    y = np.array(y)
    plot_step = 0.01
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    pyplot.contour(xx, yy, Z,colors = 'k',linewidths = 3)
    plot_points(X, y)
    pyplot.contourf(xx, yy, Z, colors=['red', 'blue'], alpha=0.2, levels=range(-1,2))
    pyplot.show()
    


In [154]:
model_C10 = SVC(C=10)
model_C10.fit(X_train, y_train)
clf_c10 = model_C10.fit(X_train, y_train)
model_C10.score(X_test, y_test)

In [155]:
model_C50 = SVC(C=50)
model_C50.fit(X_train, y_train)
clf_c50 = model_C50.fit(X_train, y_train)
model_C50.score(X_test, y_test)

In [156]:
from sklearn.model_selection import cross_val_score

In [157]:
scores1 = cross_val_score(model_C1,X,y, cv=10)
np.average(scores1)

In [158]:
scores10 = cross_val_score(model_C10,X,y, cv=10)
np.average(scores10)

In [159]:
scores50 = cross_val_score(model_C50,X,y, cv=10)
np.average(scores50)

In [160]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(SVC(gamma='auto'), {
    'C': [1,10,50],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(X_scaled,y)
clf.cv_results_

In [161]:
df = pd.DataFrame(clf.cv_results_)
df

In [162]:
df[['param_C','param_kernel','mean_test_score']]

In [163]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(), {
        'C': [1,10,50],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(X_scaled,y)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

SVM with C = 1

In [164]:
import sklearn.metrics as metrics
y_pred_c1 = model_C1.predict(X_test)
plot_points(X_test, y_pred_c1)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_c1))


SVM with C = 10

In [165]:
y_pred_c10 = model_C10.predict(X_test)
plot_points(X_test, y_pred_c10)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_c10))


SVM with C = 50

In [166]:
y_pred_c50 = model_C50.predict(X_test)
plot_points(X_test, y_pred_c50)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_c50))